# Testing Portfolio Object

## Imports 

In [1]:
# Reload Custom Modules 
import importlib
import src.beta_vae
import src.data_fetching 
import src.portfolio
importlib.reload(src.beta_vae)
importlib.reload(src.data_fetching)
importlib.reload(src.portfolio)

<module 'src.portfolio' from 'c:\\Users\\jairp\\OneDrive\\Desktop_remote\\HEC Montreal\\4. Fall 2024\\Deep Learning II\\Project\\StockDiversifier\\src\\portfolio.py'>

In [2]:
# General 
import os 
import json 
import random 
from tqdm.notebook import tqdm

# Data Science
import optuna 
import numpy as np 
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

# PyTorch
import torch

# Custom Classes 
from src.beta_vae import Encoder
from src.beta_vae import Decoder
from src.beta_vae import BetaVAE
from src.portfolio import Portfolio

# Custom Functions
from src.beta_vae import create_data_loaders
from src.beta_vae import create_single_data_loader
from src.beta_vae import objective
from src.beta_vae import train_beta_vae
from src.beta_vae import get_embeddings
from src.beta_vae import generate_embeddings_dict
from src.portfolio import fetch_and_calculate_returns
from src.portfolio import diversify_betavae_portfolio
from src.data_fetching import scrape_sp500_wikipedia
from src.data_fetching import fetch_stock_data
from src.data_fetching import prepare_data_for_vae

## Initialization

In [3]:
# Example returns for three stocks (numpy arrays of simulated returns)
returns_dict = {
    "AAPL": np.random.normal(0.001, 0.02, 1000),  # Apple
    "GOOGL": np.random.normal(0.0012, 0.025, 1000),  # Google
    "MSFT": np.random.normal(0.0008, 0.015, 1000)   # Microsoft
}

# Example weights (optional)
weights = {
    "AAPL": 0.4,
    "GOOGL": 0.4,
    "MSFT": 0.2
}

# Frequency of returns
frequency = "daily"

# Initialize the Portfolio object
portfolio = Portfolio(returns_dict=returns_dict, frequency=frequency, weights=weights, risk_free=0.00)

## Attributes

In [4]:
# Print the tickers in the portfolio
print("Tickers:", portfolio.tickers)

# Print the raw returns dictionary
print("Raw Returns:")
for ticker, returns in portfolio.raw_returns.items():
    print(f"{ticker}: {returns[:5]}...")  # Print first 5 returns for each ticker

# Print the covariance matrix
print("Covariance Matrix (Sigma):\n", portfolio.Sigma)

# Print the diagonal of the covariance matrix (sigmas)
print("Volatilities (sigmas):", portfolio.sigmas)

# Portfolio statistics
print("Portfolio Expected Return:", portfolio.portfolio_expected_return)
print("Portfolio Volatility:", portfolio.portfolio_volatility)
print("Portfolio Diversification Ratio:", portfolio.diversification_ratio)
print("Portfolio Sharpe Ratio:", portfolio.sharpe_ratio)

Tickers: ['AAPL', 'GOOGL', 'MSFT']
Raw Returns:
AAPL: [-0.00478473 -0.00550335  0.0198743  -0.00603779  0.00834376]...
GOOGL: [0.01147161 0.00214348 0.03597909 0.00415004 0.03626066]...
MSFT: [ 0.01000001  0.01412602 -0.01283901 -0.01633735  0.00747314]...
Covariance Matrix (Sigma):
 [[ 4.25156527e-04  1.34937235e-06 -3.99209384e-06]
 [ 1.34937235e-06  5.92124820e-04  2.69465549e-07]
 [-3.99209384e-06  2.69465549e-07  2.31463055e-04]]
Volatilities (sigmas): [0.02061932 0.02433362 0.01521391]
Portfolio Expected Return: 0.0007478162479712108
Portfolio Volatility: 0.01310952769749908
Portfolio Diversification Ratio: 1.6037158721613314
Portfolio Sharpe Ratio: 0.057043721576168806


## Calling Methods

In [5]:
# Get covariance matrix as a DataFrame
cov_matrix_df = portfolio.get_covariance_matrix()
print("Covariance Matrix DataFrame:\n", cov_matrix_df)

# Get expected returns and volatilities (sigmas) as dictionaries
expected_returns, sigmas = portfolio.get_expected_returns_and_sigmas()
print("Expected Returns:", expected_returns)
print("Volatilities (sigmas):", sigmas)

# Get initial weights
init_weights = portfolio.get_weights()

# Optimize weights for maximum diversification
optimized_weights = portfolio.optimize_weights(method="max_div", update_weights=False)

# Update weights in the portfolio object after optimization
portfolio.optimize_weights(method="max_div", update_weights=True)

# Show weights before and after optimization 
print("Initial Weights:", init_weights)
print("Updated Weights:", portfolio.get_weights())

# Updated portfolio statistics after optimization
print("Updated Portfolio Expected Return:", portfolio.portfolio_expected_return)
print("Updated Portfolio Volatility:", portfolio.portfolio_volatility)
print("Updated Portfolio Diversification Ratio:", portfolio.diversification_ratio)
print("Updated Portfolio Sharpe Ratio:", portfolio.sharpe_ratio)

Covariance Matrix DataFrame:
            AAPL         GOOGL          MSFT
AAPL   0.000425  1.349372e-06 -3.992094e-06
GOOGL  0.000001  5.921248e-04  2.694655e-07
MSFT  -0.000004  2.694655e-07  2.314631e-04
Expected Returns: {'AAPL': 0.000517199874991859, 'GOOGL': 0.0014202351336783325, 'MSFT': -0.00013578877748432884}
Volatilities (sigmas): {'AAPL': 0.02061932412035988, 'GOOGL': 0.024333615030209324, 'MSFT': 0.015213909923950938}
Initial Weights: {'AAPL': 0.4, 'GOOGL': 0.4, 'MSFT': 0.2}
Updated Weights: {'AAPL': 0.31282085540222826, 'GOOGL': 0.2619224390465119, 'MSFT': 0.42525670555125983}
Updated Portfolio Expected Return: 0.00047603726937763783
Updated Portfolio Volatility: 0.011104224584045287
Updated Portfolio Diversification Ratio: 1.7374911211010833
Updated Portfolio Sharpe Ratio: 0.04286992448456195


## Copying and Updating The Portfolio 

In [6]:
# Example returns for additional stocks
returns_dict_new = {
    "AMZN": np.random.normal(0.0015, 0.03, 1000),  # Amazon
    "TSLA": np.random.normal(0.002, 0.05, 1000)   # Tesla
}

# Copy the existing portfolio
copied_portfolio = portfolio.copy_portfolio()
print("\n--- Copied Portfolio ---")
print("Tickers in copied portfolio:", copied_portfolio.tickers)
print("Weights in copied portfolio:", copied_portfolio.get_weights())
print("Expected Return (Copied):", copied_portfolio.portfolio_expected_return)
print("Volatility (Copied):", copied_portfolio.portfolio_volatility)
print("Diversification Ratio (Copied):", copied_portfolio.diversification_ratio)
print("Sharpe Ratio (Copied):", copied_portfolio.sharpe_ratio)

# Update portfolio: Add new tickers and remove an existing ticker
add_tickers = {"AMZN": returns_dict_new["AMZN"], "TSLA": returns_dict_new["TSLA"]}
remove_tickers = ["MSFT"]
new_weights = {"AAPL": 0.3, "GOOGL": 0.3, "AMZN": 0.2, "TSLA": 0.2}

# Update the portfolio 
portfolio.update_portfolio(add_tickers=add_tickers, remove_tickers=remove_tickers, new_weights=new_weights)

# Print updated portfolio attributes
print("\n--- Updated Portfolio ---")
print("Updated Tickers:", portfolio.tickers)
print("Updated Weights:", portfolio.get_weights())

# Portfolio statistics after update
print("Updated Portfolio Expected Return:", portfolio.portfolio_expected_return)
print("Updated Portfolio Volatility:", portfolio.portfolio_volatility)
print("Updated Portfolio Diversification Ratio:", portfolio.diversification_ratio)
print("Updated Portfolio Sharpe Ratio:", portfolio.sharpe_ratio)

# Optimize updated portfolio weights and compare again 
print("\n--- Optimization After Update ---")
portfolio.optimize_weights(method="max_div", update_weights=True)

# Updated portfolio statistics after optimization
print("Updated Tickers:", portfolio.tickers)
print("Updated Weights:", portfolio.get_weights())
print("Updated Portfolio Expected Return:", portfolio.portfolio_expected_return)
print("Updated Portfolio Volatility:", portfolio.portfolio_volatility)
print("Updated Portfolio Diversification Ratio:", portfolio.diversification_ratio)
print("Updated Portfolio Sharpe Ratio:", portfolio.sharpe_ratio)



--- Copied Portfolio ---
Tickers in copied portfolio: ['AAPL', 'GOOGL', 'MSFT']
Weights in copied portfolio: {'AAPL': 0.31282085540222826, 'GOOGL': 0.2619224390465119, 'MSFT': 0.42525670555125983}
Expected Return (Copied): 0.00047603726937763783
Volatility (Copied): 0.011104224584045287
Diversification Ratio (Copied): 1.7374911211010833
Sharpe Ratio (Copied): 0.04286992448456195

--- Updated Portfolio ---
Updated Tickers: ['AAPL', 'GOOGL', 'AMZN', 'TSLA']
Updated Weights: {'AAPL': 0.3, 'GOOGL': 0.3, 'AMZN': 0.2, 'TSLA': 0.2}
Updated Portfolio Expected Return: 0.0009028904663642618
Updated Portfolio Volatility: 0.014656098658801162
Updated Portfolio Diversification Ratio: 2.0057103946228163
Updated Portfolio Sharpe Ratio: 0.04286992448456195

--- Optimization After Update ---
Updated Tickers: ['AAPL', 'GOOGL', 'AMZN', 'TSLA']
Updated Weights: {'AAPL': 0.3448536665254034, 'GOOGL': 0.2735961468878023, 'AMZN': 0.22716833582296084, 'TSLA': 0.15438185076383346}
Updated Portfolio Expected Re